# Baseline Models

- Using three baseline models for the development of an analysis that detects political language in court documents.
    1. Logistic regression
    2. Naive Bayes
    3. SVM
- Using baselines as a means for testing model performance and building an accurate, model for classifying language used in Bulgaria's constitutional court

## Next steps

- Data needs more sentences labelled as political, as data is imbalanced and models observe few political sentences
- Need to optimize hyperparameters to improve model performance

In [5]:
import glob
import os
import pickle
import re

import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn import naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split


In [6]:
# load all json files in data directory
files = glob.glob("data/json/*.json")
data = []

# for loop for processing files and adding doc id
for json in files:
    frame = pd.read_json(json)
    # get file name as string
    # create column identifying dfs as doc_id
    # split string (remove .json from file name)
    frame["doc_id"] = os.path.splitext(os.path.basename(json))[0]
    data.append(frame)

# concatenate all data frames
df = pd.concat(data, ignore_index=True)

## Cleaning Text

In [7]:
# create binary variable where POLITICAL = 1, all else = 0
df.loc[df["label_id"] != 4, "label_id"] = 0

df.loc[df["label_id"] == 4, "label_id"] = 1


In [8]:
def preprocessing(text):

    text = re.sub('<[^>]*>', '', text)
    text = re.sub(r'[^\w\s]','', text)
    stop_words = set(stopwords.words("english"))
    words = [word for word in text.lower().split() if not word in stop_words]
    text = " ".join(words)

    return text

In [9]:
df['text'] = df['text'].apply(preprocessing)

In [11]:
ps = PorterStemmer()

def token_ps(text):
    return [ps.stem(word) for word in text.split()]

## Logistic Regression

- Computing a logistic regression model based on the values created from a vectorizer algorithm called tf-idf, which stands for term-frequency inverse document frequency.
- tf-idf measures the originality of the word by comparing how often it appears in a doc with the number of docs the word appears in. The frequency of the words in a doc (compared against other docs) measures the importance of that word in the wider corpus.
- The logistic regression below is computed by building a vector of word values based on the iportance of each word, before using the word vectors to identify the characteristics of the political label to predict which sentences will be political.

In [12]:
# transforming text into vectors
tfidf = TfidfVectorizer(lowercase=False,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)
# compute tfidf values for all words in 'text' column of df
X = tfidf.fit_transform(df['text'])
y = df.label_id.values

In [13]:
# splitting data into train and test splits in order to test predictive accuracy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0, shuffle=True
)

# computes and then fits logistic regression that implements cross-validation as a part of the process
# cv = number of cross validation folds
log_reg = LogisticRegressionCV(
    cv=8, scoring="accuracy", n_jobs=-1, verbose=3
).fit(X_train, y_train)

# model accuracy
log_predictions = log_reg.predict(X_test)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:    4.1s remaining:   12.3s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    4.8s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    5.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    5.1s finished


In [14]:
# defining a function that prints model prediction accuracy
def model_accuracy(name, preds):
    print("---{} Test Set Results---".format(name))
    print("Weighted F1 Average: {}".format(f1_score(y_test, preds, average="weighted")))
    # precision = % predicted accurately
    # recall = % positives identified
    # f1-score = weighted harmonic mean of precision & recall
    # weighted f-1 avg used for comparing classification models
    print(classification_report(y_test, preds))

In [15]:
model_accuracy("Logit", log_predictions)

---Logit Test Set Results---
Weighted F1 Average: 0.8000997426850943
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       127
           1       1.00      0.05      0.09        22

    accuracy                           0.86       149
   macro avg       0.93      0.52      0.51       149
weighted avg       0.88      0.86      0.80       149



## Naive Bayes

In [16]:
# fit the training dataset on the NB classifier
nb = naive_bayes.ComplementNB()
nb.fit(X_train, y_train)

# model accuracy
nb_predictions = nb.predict(X_test)
model_accuracy("Naive Bayes", nb_predictions)

---Naive Bayes Test Set Results---
Weighted F1 Average: 0.8372325419305285
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       127
           1       0.47      0.36      0.41        22

    accuracy                           0.85       149
   macro avg       0.68      0.65      0.66       149
weighted avg       0.83      0.85      0.84       149



## Support Vector Machines (SVM)

In [17]:
# fit the training dataset on the SVM classifier
SVM = svm.SVC(C=1, kernel='linear', class_weight='balanced')
SVM.fit(X_train, y_train)

# model accuracy
svm_predictions = SVM.predict(X_test)
model_accuracy("SVM", svm_predictions)

---SVM Test Set Results---
Weighted F1 Average: 0.8147422394225902
              precision    recall  f1-score   support

           0       0.86      1.00      0.93       127
           1       1.00      0.09      0.17        22

    accuracy                           0.87       149
   macro avg       0.93      0.55      0.55       149
weighted avg       0.88      0.87      0.81       149



## Pickling Models (for Future Use)

In [18]:
# saving tfidf
pickle.dump(tfidf, open('tfidf.pickle', 'wb'))

# saving models
pickle.dump(log_reg, open('log_reg.pickle', 'wb'))
pickle.dump(nb, open('nb.pickle', 'wb'))
pickle.dump(SVM, open('svm.pickle', 'wb'))